In [10]:
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import  RandomUnderSampler, RepeatedEditedNearestNeighbours
from sklearn.datasets import make_classification
pd.set_option("display.max_columns", 100)
from sklearn.preprocessing import KBinsDiscretizer
DATA_PATH = Path.cwd().parent / "ML project" 
import math
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.impute import MissingIndicator
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
import eli5
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import f1_score
from sklearn.multioutput import MultiOutputClassifier
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, roc_auc_score
import xgboost as xgb
from itertools import combinations
RANDOM_SEED = 6    # Set a random seed for reproducibility!
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from xgboost import XGBClassifier

In [5]:

from math import cos, asin, sqrt, pi

def distanceLong(lat1, lon1, lat2, lon2):
    p = pi/180
    latDiff = (lat2-lat1)*p
    lonDiff = (lon2-lon1)*p
    coslat1 = np.cos(lat1*p)
    coslat2 = np.cos(lat2*p)
    coslondiff = np.cos(lonDiff)
    coslatdiff = np.cos(latDiff)
    a = 0.5 - coslatdiff/2 + coslat1 * coslat2 * (1-coslondiff)/2
    return 12742 * (np.arcsin(a**0.5)) 

In [6]:

def formatdata(dataFrame):
    dataFrame['pickup_time'] = dataFrame['pickup_time']+ '.099'
    dataFrame['pickup_time'] = dataFrame['pickup_time'].apply(lambda x:  x.replace("/","-"))
    dataFrame['drop_time'] = dataFrame['drop_time']+ '.099'

    pd.to_datetime(dataFrame['pickup_time'], format='%m-%d-%Y %H:%M:%S.%f', errors='ignore')
    pd.to_datetime(dataFrame['drop_time'], format='%m/%d/%Y %H:%M:%S.%f', errors='ignore')
    
    dataFrame['pickup_time'] = dataFrame['pickup_time'].apply(lambda x: datetime.timestamp(datetime.strptime(x,'%m-%d-%Y %H:%M.%f')))
    dataFrame['drop_time'] = dataFrame['drop_time'].apply(lambda x: datetime.timestamp(datetime.strptime(x,'%m/%d/%Y %H:%M.%f')))

    dataFrame['timediff'] =  dataFrame['drop_time'] - dataFrame['pickup_time']
    #dataFrame['timedifferant'] =  dataFrame['drop_time'] - dataFrame['pickup_time'] + dataFrame['duration']
    dataFrame['distance'] = distanceLong(dataFrame['drop_lat'],dataFrame['drop_lon'],dataFrame['pick_lat'],dataFrame['pick_lon'])*100
    #dataFrame['distance'] =  ((dataFrame['drop_lat']-dataFrame['pick_lat'])**2+(dataFrame['drop_lon']-dataFrame['pick_lon'])**2)**0.5
    #dataFrame['unit_meter_waiting_fair'] = dataFrame['meter_waiting_fare'] / dataFrame['meter_waiting']
    #print(dataFrame['unit_meter_waiting_fair'])
    
    #dataFrame['_fare'] = dataFrame['fare'] - dataFrame['additional_fare'] 
    #dataFrame['tripDuration'] = dataFrame['duration'] - dataFrame['meter_waiting']
    
    
    return dataFrame

    

In [1]:
def formatFeatures(dataFrame, datatype = 0):
    df = dataFrame
    
    all_features = df.columns.values
    
    df.replace({999.0: np.NaN}, inplace=True)
    indicator = MissingIndicator(missing_values=np.NaN)
    indicator = indicator.fit_transform(df)
    
    indicator = pd.DataFrame(indicator,columns=["ind"+str(i) for i in range(indicator.shape[1])], index=df.index)
    
    imp_numerical = SimpleImputer(strategy='mean',missing_values=np.nan)
    imp_catagorical = SimpleImputer(strategy='constant',missing_values=np.nan)
    
    df[df.columns[df.dtypes != "object"].values] = imp_numerical.fit_transform(df[df.columns[df.dtypes != "object"].values])
    #df[df.columns[df.dtypes == "object"].values] = imp_catagorical.fit_transform(df[df.columns[df.dtypes == "object"].values])
    
    numerical_features = df.columns[df.dtypes != "object"]
    allColsToPoly = df.columns.values
    polyList = [['additional_fare','meter_waiting_fare'],['additional_fare', 'fare'],
                ['distance','timediff'],['timediff','meter_waiting'],['timediff','duration'],['fare','timediff']]
    poly = PolynomialFeatures(degree=3, interaction_only=True)
    #polynomials = pd.DataFrame(poly
     #                          .fit_transform(df[]),
      #                         columns=["poly"+str(i) for i in range(4)],index=df.index)
    
    #polynomials = polynomials.drop(['poly0'],axis=1)
    df = pd.concat([df, indicator], axis=1)
    for i in polyList:
                
        colName = ''
        for j in i:
            colName +=j+"_"
        poly = PolynomialFeatures(degree=3, interaction_only=True)
        polynomials = pd.DataFrame(poly
                                    .fit_transform(df[list(i)]),
                                    columns=[colName+str(i) for i in range(4)],index=df.index)

        polynomials = polynomials.drop([colName+'0'],axis=1) 
        df = pd.concat([df, polynomials], axis=1)
   
    #continuesList = ['duration','meter_waiting','meter_waiting_fare','meter_waiting_till_pickup','fare']
    #continuesList=[]
    #for i in continuesList:
     #   disc = KBinsDiscretizer(n_bins=3, encode='ordinal', 
      #                          strategy='uniform')
        #df[i] = disc.fit_transform(df[i].values.reshape(-1,1))
    
    allCols = df.columns.values
    for i in allCols:
        scaler = StandardScaler()
        df[i] = scaler.fit_transform(df[i].values.reshape(-1,1))
    features_df_norm_l2 = list(math.sqrt(sum(list((i**2) for i in df.iloc[r]))) 
                           for r in range(len(df)))
    
    indexCount = 0
    for index, row in df.iterrows():
        rowt = row/features_df_norm_l2[indexCount]
        df.loc[index] = rowt
    return df

In [2]:
train_df = pd.read_csv(
    DATA_PATH / "train.csv", 
    index_col="tripid"
)
train_df = train_df.drop_duplicates()


train_df['label'] = train_df['label'].map({"correct":1, "incorrect":0})
train_df_cols = train_df.columns.values
li = list(train_df_cols)
features_df = train_df[li[0:-1]]
labels_df = train_df[li[-1]]



features_df = formatdata(features_df)

features_df = features_df.drop(['drop_lon','pick_lon','drop_lat','pick_lat','pickup_time','drop_time'],axis=1)
features_df = formatFeatures(features_df)


numeric_cols = features_df.columns[features_df.dtypes != object].values

numeric_preprocessing_steps = Pipeline([
    ('standard_scaler', StandardScaler()),
    ('simple_imputer', SimpleImputer(strategy='most_frequent'))
])
#categorical_transformer = Pipeline(steps=[
 #   ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
  #  ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# create the preprocessor stage of final pipeline
# each entry in the transformer list is a tuple of
# (name you choose, sklearn transformer, list of columns)
preprocessor = ColumnTransformer(
    transformers = [
        ("numeric", numeric_preprocessing_steps, numeric_cols)       
    ],
    remainder = "drop"
)









NameError: name 'pd' is not defined

In [18]:
solver = 'lbfgs'
alpha = 2
hidden_layer_sizes=(7,2 )
random_state=2
max_iter= 100000000

activation='tanh'
#estimators=MLPClassifier(solver=solver, alpha=alpha, hidden_layer_sizes=hidden_layer_sizes, random_state=random_state,
 #                        max_iter=max_iter, activation=activation)
min_child_weight=2
max_depth=5
gamma=0.02
colsample_bytree=0.8
subsample=0.8
n_estimators=200
threads = 8
estimators = XGBClassifier(min_child_weight=min_child_weight,max_depth=max_depth, 
                           gamma=gamma,colsample_bytree=colsample_bytree,subsample=subsample,
                           n_estimators=n_estimators, n_jobs=threads
                          )
#estimators = BaggingClassifier(base_estimator=estimatorsY,n_estimators=10,max_samples=1.0,
 #                            max_features=1.0, random_state=1,bootstrap=False)

X_train, X_eval, y_train, y_eval = train_test_split(
    features_df,
    labels_df,
    test_size=0.33,
    shuffle=True,
    stratify=labels_df,
    random_state=2
)
full_pipelinea = Pipeline([
    
    ("estimators", estimators),
])

estimators.fit(X_train, y_train)
preds = estimators.predict(X_eval)
y_preds = pd.DataFrame(
    {'prediction':preds},
    index = y_eval.index
)

f1 = f1_score(y_eval, y_preds)
print(y_preds.prediction.value_counts())
print(f1)

1    5275
0     392
Name: prediction, dtype: int64
0.9744922514197709


In [365]:
test_features_df = pd.read_csv(DATA_PATH / "test.csv", 
                               index_col="tripid")

test_features_df = formatdata(test_features_df)
test_features_df = test_features_df.drop(['drop_lon','pick_lon','drop_lat','pick_lat','pickup_time','drop_time'],axis=1)
test_features_df = formatFeatures(test_features_df)
test_features_df['ind2'] = False
test_features_df['ind3'] = False
test_features_df['ind4'] = False
test_features_df['ind5'] = False
test_probas = estimators.predict(test_features_df)
test_preds = pd.DataFrame(
    {'prediction':test_probas},
    index = test_features_df.index
)

print(test_preds.prediction.value_counts())

1    5464
0    3112
Name: prediction, dtype: int64


In [348]:
print(test_preds.prediction.value_counts())
print(f1)

1    8031
0     545
Name: prediction, dtype: int64
0.9524279023909293


In [355]:
test_preds.to_csv(DATA_PATH / 'my_submission.csv', index=True)

In [186]:
def modelfit(alg, xtrain,ytrain, predictors, title,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    X_train, X_eval, y_train, y_eval = train_test_split(
        xtrain,
        ytrain,
        test_size=0.33,
        shuffle=True,
        stratify=ytrain,
        random_state=6
    )
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(xtrain[predictors].values, label=ytrain.values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    
    alg.fit(X_train[predictors], y_train,eval_metric='auc')
        
    #Predict training set:
    dtrain_pred = alg.predict(X_train[predictors])
    deval_pred = alg.predict(X_eval[predictors])
    #Print model report:
    print("\nModel Report: "+title)
    print ("Accuracy : %.4g" % metrics.accuracy_score(y_train, dtrain_pred))
    print ("f1 Score (Train): %f" % metrics.f1_score(y_train, dtrain_pred))
    print ("f1 Score (Test): %f" % metrics.f1_score(y_eval, deval_pred))
    feat_imp = pd.Series(alg.feature_importances_)
    feat_imp.plot(kind='bar', title=title)
    plt.ylabel('Feature Importance Score')
    plt.show()

In [187]:
predictors = features_df.columns.values
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=100,
 max_depth=6,
 min_child_weight=3,
 gamma=0.1,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=8,
 n_jobs=8,
 scale_pos_weight=1,
 seed=27)

#modelfit(xgb1, features_df, labels_df, predictors, 'fareinfare')


In [ ]:
colList = list(features_df.columns.values)
for i in range(len(colList)):
    print (i, colList[i])

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from lightgbm import LGBMClassifier

#estimators = MultiOutputClassifier(
 #   estimator=MLPClassifier(solver='lbfgs', alpha=.55, hidden_layer_sizes=(4,2 ), random_state=1, max_iter=10000000)
#)
#estimators = MultiOutputClassifier(
 #   estimator=SVC(gamma='auto', probability=True),
#)
#estimators =SVC(gamma='auto')
#estimatorsY=LogisticRegression(penalty="l2", C=20,solver='lbfgs',max_iter=1000000)
solver = 'lbfgs'
alpha = 0.075
hidden_layer_sizes=(7,2 )
random_state=80
max_iter= 10000000

activation='tanh'
estimators=MLPClassifier(solver=solver, alpha=alpha, hidden_layer_sizes=hidden_layer_sizes, random_state=random_state,
                         max_iter=max_iter, activation=activation)


#estimatorsX2 = GradientBoostingClassifier(n_estimators=20, learning_rate=0.5, max_features=2, max_depth=1, random_state=0)
min_child_weight=2
max_depth=5
gamma=0.5
colsample_bytree=0.75
subsample=0.75
n_estimators=5000
threads = 8
#estimators = XGBClassifier(min_child_weight=min_child_weight,max_depth=max_depth, 
 #                          gamma=gamma,colsample_bytree=colsample_bytree,subsample=subsample,
  #                         n_estimators=n_estimators, n_jobs=threads
   #                       )
#params = { 'axgb__n_estimators': [20, 200],'cmlp__alpha':[0,1.0],'blr__C': [1.0, 100.0]}
#estimatorsX3 = AdaBoostClassifier(n_estimators=100, random_state=0, learning_rate = 0.05, 
#                                algorithm='SAMME.R',base_estimator=estimatorsX)

#estimators = RandomForestClassifier(n_estimators=50, criterion='entropy',max_depth=10,min_samples_split=4,
 #                                  min_samples_leaf=4,max_features='log2',max_leaf_nodes=100,bootstrap=True,
  #                                  oob_score=True,class_weight='balanced_subsample')
#estimatorNB = GaussianNB()

#estimatorsY = LGBMClassifier(boosting_type='gbdt', num_leaves=40, max_depth=6,learning_rate=0.2,
 #                          n_estimators=1000,random_state=42)

#estimators = KNeighborsClassifier(n_neighbors=5, weights='distance',algorithm='brute',p=2 ,leaf_size=10)
#estimators = BaggingClassifier(base_estimator=estimatorsY,n_estimators=10,max_samples=1.0,
 #                            max_features=1.0, random_state=1,bootstrap=False)
#estimators = VotingClassifier( [('xgb', estimatorsX), ('mlp', estimatorsY)], weights=[1,1])
#estimators = GridSearchCV(estimator=vestimators, param_grid=params, cv=5)
#estimators = HistGradientBoostingClassifier(max_iter=500,min_samples_leaf=2,max_depth=10, learning_rate=0.08)


In [ ]:
solver = 'lbfgs'
alpha = 0.075
hidden_layer_sizes=(7,2 )
random_state=2
max_iter= 10000000

activation='tanh'
estimators=MLPClassifier(solver=solver, alpha=alpha, hidden_layer_sizes=hidden_layer_sizes, random_state=random_state,
                         max_iter=max_iter, activation=activation)
min_child_weight=2
max_depth=5
gamma=0.01
colsample_bytree=0.75
subsample=0.75
n_estimators=165
threads = 8
estimatorsY = XGBClassifier(min_child_weight=min_child_weight,max_depth=max_depth, 
                           gamma=gamma,colsample_bytree=colsample_bytree,subsample=subsample,
                           n_estimators=n_estimators, n_jobs=threads
                          )
#estimators = BaggingClassifier(base_estimator=estimatorsY,n_estimators=10,max_samples=1.0,
 #                            max_features=1.0, random_state=1,bootstrap=False)

X_train, X_eval, y_train, y_eval = train_test_split(
    features_df,
    labels_df,
    test_size=0.20,
    shuffle=True,
    stratify=labels_df,
    random_state=6
)
full_pipelinea = Pipeline([
    ("preprocessor", preprocessor),
    ("estimators", estimators),
])

estimators.fit(X_train, y_train)
preds = estimators.predict(X_eval)
y_preds = pd.DataFrame(
    {'prediction':preds},
    index = y_eval.index
)

f1 = f1_score(y_eval, y_preds)
print(f1)

In [ ]:
test_features_df = pd.read_csv(DATA_PATH / "test.csv", 
                               index_col="tripid")

test_features_df = formatdata(test_features_df)
test_features_df = test_features_df.drop(['drop_lon','pick_lon','drop_lat','pick_lat','pickup_time','drop_time'],axis=1)
test_features_df = formatFeatures(test_features_df)
test_probas = estimators.predict(test_features_df)
test_preds = pd.DataFrame(
    {'prediction':test_probas},
    index = test_features_df.index
)
print(test_preds.prediction.value_counts())

In [41]:
print(test_preds.prediction.value_counts())

1    8345
0     231
Name: prediction, dtype: int64


In [86]:
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
bestModel = ''
bestscore = 0
scores = []
random_s = 0
for i in range(80,81):
    
    kf = KFold(n_splits=5, random_state=i, shuffle=True)

    kf.get_n_splits(features_df)



    for train_index, test_index in kf.split(features_df):
        X_train, X_eval, y_train, y_eval = features_df.iloc[train_index], features_df.iloc[test_index],labels_df.iloc[train_index], labels_df.iloc[test_index]
        
        estimatorsY = XGBClassifier(min_child_weight=min_child_weight,max_depth=max_depth, 
                                   gamma=gamma,colsample_bytree=colsample_bytree,subsample=subsample,
                                   n_estimators=n_estimators, n_jobs=threads
                                  )
        estimators = BaggingClassifier(base_estimator=estimatorsY,n_estimators=10,max_samples=1.0,
                                     max_features=1.0, random_state=1,bootstrap=False)
        estimators.fit(X_train, y_train)
        preds = estimators.predict(X_eval)
        y_preds = pd.DataFrame(
            preds,
            index = y_eval.index
        )

        f1 = f1_score(y_eval, y_preds)
        scores.append(f1)
        if(f1 > bestscore):
            bestscore = f1
            bestModel = estimators
            random_s = i
        print(f1)

print('bestscore', bestscore)
print('random state', random_s)

0.975378787878788
0.9759036144578315
0.9735466497703152
0.9700541573749603
0.9721647844759027
bestscore 0.9759036144578315
random state 80


In [ ]:
print('bestscore',bestscore)

In [87]:
test_features_df = pd.read_csv(DATA_PATH / "test.csv", 
                               index_col="tripid")

test_features_df = formatdata(test_features_df)
test_features_df = test_features_df.drop(['drop_lon','pick_lon','drop_lat','pick_lat','pickup_time','drop_time'],axis=1)
test_features_df = formatFeatures(test_features_df)
test_probas = bestModel.predict(test_features_df)
test_preds = pd.DataFrame(
    {'prediction':test_probas},
    index = test_features_df.index
)

In [91]:
test_probas = bestModel.predict(test_features_df)

test_probas

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [92]:
test_preds = pd.DataFrame(
    {'prediction':test_probas},
    index = test_features_df.index
)

print(test_preds['prediction'].max())
print(test_preds['prediction'].min())

1
0


In [93]:
test_preds.to_csv(DATA_PATH / 'my_submission.csv', index=True)

In [ ]:
submission_df.head()

In [ ]:
# Make sure we have the rows in the same order
np.testing.assert_array_equal(test_features_df.index.values, 
                              submission_df.index.values)

# Save predictions to submission data frame
submission_df["h1n1_vaccine"] = test_preds[1]

submission_df.head()

In [ ]:
submission_df.to_csv(DATA_PATH / 'my_submission.csv', index=True)

In [ ]:
from datetime import datetime
to = '11/01/2019  00:20:00'
to += '.099'
froms = '11/01/2019  00:34:00'+'.099'
dateObjectto = datetime.strptime(to, '%m/%d/%Y %H:%M:%S.%f')
dateObjectfrom = datetime.strptime(froms, '%m/%d/%Y %H:%M:%S.%f')
print(datetime.timestamp(dateObjectfrom) - datetime.timestamp(dateObjectto))


In [ ]:
train_df = pd.read_csv(
    DATA_PATH / "train.csv", 
    index_col="tripid"
)
train_df['label'] = train_df['label'].map({"correct":1, "incorrect":0})
train_df_cols = train_df.columns.values
li = list(train_df_cols)
features_df = train_df[li[0:-1]]
labels_df = train_df[li[-1]]

In [ ]:
features_df.describe()

In [ ]:
print(features_df.dtypes)

In [ ]:
import matplotlib.pyplot as plt

features_df[features_df.dtypes[(features_df.dtypes=="float64")]
                        .index.values].hist(figsize=[11,11],
                                            range=[features_df['drop_lat'].min(), features_df['drop_lat'].max()])


In [ ]:
from sklearn import preprocessing

cols_to_norm = ['duration','meter_waiting',
                'meter_waiting_fare','meter_waiting_till_pickup',
                'pick_lat','pick_lon','drop_lat','drop_lon','fare']
n_test = features_df[cols_to_norm]


x = n_test.values

min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)

n_test = pd.DataFrame(x_scaled, columns=cols_to_norm,index=features_df.index)

l_test = features_df.drop(cols_to_norm, axis=1)
train = pd.concat([n_test, l_test], axis=1)
train.columns

In [ ]:
from matplotlib import pyplot as plt

def plot():
    plt.figure(figsize=(8,6))

    plt.scatter(features_df['meter_waiting_fare'], labels_df,
            color='green', label='input scale', alpha=0.5)

    
    plt.xlabel('Alcohol')
    plt.ylabel('Malic Acid')
    plt.legend(loc='upper left')
    plt.grid()

    plt.tight_layout()

    
plot()
plt.show()

In [ ]:
import seaborn as sns
train_df = pd.read_csv(
    DATA_PATH / "train.csv", 
    index_col="tripid"
)
train_df['label'] = train_df['label'].map({"correct":1, "incorrect":0})
train_df_cols = train_df.columns.values
li = list(train_df_cols)
features_df = train_df[li[0:-1]]
labels_df = train_df[li[-1]]
features_df = formatdata(features_df)
features_df = features_df.join(labels_df)

labels_df.fillna(0)
features_df[features_df==np.inf]=np.nan
features_df.fillna(0, inplace=True)
numericOnle = ['duration','meter_waiting','meter_waiting_fare','meter_waiting_till_pickup','fare','timediff','distance','label']
features_set = features_df[numericOnle]

#get correlations of each features in dataset
corrmat = features_set.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(8,8))
#plot heat map
g=sns.heatmap(features_set[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [ ]:
# print(__doc__)

# Author: Mathieu Blondel
#         Jake Vanderplas
# License: BSD 3 clause

import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline


def f(x):
    """ function to approximate by polynomial interpolation"""
    return x * np.sin(x)


X, Y = features_df, labels_df
X_train, X_eval, y_train, y_eval = train_test_split(
    features_df,
    labels_df,
    test_size=0.33,
    shuffle=True,
    stratify=labels_df,
    random_state=6
)
colors = ['teal', 'yellowgreen', 'gold']
lw = 2
plt.plot(X, f(X), color='cornflowerblue', linewidth=lw,
         label="ground truth")
plt.scatter(X_train['timediff'], y_train, color='navy', s=20, marker='o', label="training points")

for count, degree in enumerate([3, 4, 5]):
    model = make_pipeline(PolynomialFeatures(degree), Ridge())
    model.fit(X_train, y_train)
    y_plot = model.predict(X_eval)
    plt.plot(X_train, y_train, color=colors[count], linewidth=lw,
             label="degree %d" % degree)

plt.legend(loc='lower left')

plt.show()